In [1]:
from filter_functions import image_filter

2022-08-18 14:45:21,801 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-ehygh630', purging
2022-08-18 14:45:21,801 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-4zgswl0v', purging
2022-08-18 14:45:21,801 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-7a9f4vx5', purging
2022-08-18 14:45:21,801 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-9jcgnr2g', purging


Dashboard at http://127.0.0.1:8787/status


In [2]:
from training_functions import train_segmentation

In [3]:
# import dask
# import xarray as xr
import numpy as np
import  matplotlib.pyplot as plt
import os
from ipywidgets import Image
from ipywidgets import ColorPicker, IntSlider, link, AppLayout, HBox
from ipycanvas import  hold_canvas,  MultiCanvas #RoughCanvas,Canvas,
import imageio

In [4]:
def on_mouse_down(x, y):
    global drawing
    global position
    global shape
    drawing = True
    position = (x, y)
    shape = [position]

def on_mouse_move(x, y):
    global drawing
    global position
    global shape
    if not drawing:
        return
    with hold_canvas():
        canvas.stroke_line(position[0], position[1], x, y)
        position = (x, y)
    shape.append(position)

def on_mouse_up(x, y):
    global drawing
    global positiondu
    global shape
    drawing = False
    with hold_canvas():
        canvas.stroke_line(position[0], position[1], x, y)
        canvas.fill_polygon(shape)
    shape = []

In [5]:
path = '/home/fische_r/NAS/testing/Jeremy_tomo/tomodata.nc'
# path = r"C:\Zwischenlager\tomodata.nc"

In [6]:
IF = image_filter(data_path=path)

In [7]:
IF.open_raw_data()

In [8]:
# IF.data

In [9]:
IF.prepare()

In [10]:
IF.stack_features()

In [11]:
# IF.feature_stack

In [12]:
IF.make_xarray_nc()

In [13]:
# IF.result

In [14]:
# IF.compute()

In [15]:
# IF.feature_stack.persist()

In [16]:
# test = IF.result['feature_stack'].sel(x=0,time=0).data

In [17]:
# test

In [18]:
# training_path = r"C:\Zwischenlager\Jeremy_tomo"
training_path = '/home/fische_r/NAS/testing/Jeremy_tomo'
if not os.path.exists(training_path):
    os.mkdir(training_path)

In [19]:
TS = train_segmentation(training_path=training_path)

In [20]:
TS.import_lazy_feature_stack(IF.result)

### iterative training, if you have a training set, skip here

In [21]:
TS.suggest_training_set()

You could try  time = 5  and  y = 15
However, please sort it like the original xyztime


In [22]:
c1 = 'y'
p1 = 10
c2 = 'time'
p2 = 0

In [23]:
TS.load_training_set(c1, p1, c2, p2)

existing label set loaded


/mnt/nas_Uwrite/fische_r/lib/pytrainseg/training_functions.py:176: RuntimeWarning: invalid value encountered in divide
  diff = diff/diff.max()*255


In [24]:
# TS.interface()

In [25]:
# TS.current_im8 = TS.current_im8.compute()

In [26]:
alpha = 0.15
im8 = TS.current_im8
resultim = TS.current_result
width = im8.shape[1]
height = im8.shape[0]
Mcanvas = MultiCanvas(4, width=width, height=height)
background = Mcanvas[0]
resultdisplay = Mcanvas[2]
truthdisplay = Mcanvas[1]
canvas = Mcanvas[3]
canvas.sync_image_data = True
drawing = False
position = None
shape = []
image_data = np.stack((im8, im8, im8), axis=2)
background.put_image_data(image_data, 0, 0)
resultdisplay.global_alpha = alpha
if np.any(resultim>0):
    result_data = np.stack((255*(resultim==0), 255*(resultim==1), 255*(resultim==2)), axis=2)
else:
    result_data = np.stack((0*resultim, 0*resultim, 0*resultim), axis=2)
resultdisplay.put_image_data(result_data, 0, 0)
canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)
picker = ColorPicker(description="Color:", value="#ff0000")
slidealpha = IntSlider(description="Result overlay", value=0.15)
link((picker, "value"), (canvas, "stroke_style"))
link((picker, "value"), (canvas, "fill_style"))
HBox((Mcanvas, picker, slidealpha))

In [27]:
label_set = canvas.get_image_data()

TS.current_truth[label_set[:,:,0]>0] = 1
TS.current_truth[label_set[:,:,1]>0] = 2
TS.current_truth[label_set[:,:,2]>0] = 4

imageio.imsave(TS.current_truthpath, TS.current_truth)

In [28]:
TS.train_slice()

now actually calculating the features
training and classifying


### when done, maybe save the classifier

In [38]:
# TS.pickle_classifier()

### use an existing trainingset to train classifier (adhere to label iamge naming convention)

In [ ]:
# provide new feature data if necessary and say if it is a lazy dask array or not
# TS.feat_data = 
# TS.lazy = 

TS.train()

Need to actually calculate the features ...
Need to actually calculate the features ...
